In [1]:
import numpy as np
import pyphi

from qutip import *
from qutip_qip.operations import *
from intrinsic_difference import intrinsic_difference
from utils import *
from compute_ces import *


Welcome to PyPhi!

If you use PyPhi in your research, please cite the paper:

  Mayner WGP, Marshall W, Albantakis L, Findlay G, Marchman R, Tononi G.
  (2018). PyPhi: A toolbox for integrated information theory.
  PLOS Computational Biology 14(7): e1006343.
  https://doi.org/10.1371/journal.pcbi.1006343

Documentation is available online (or with the built-in `help()` function):
  https://pyphi.readthedocs.io

To report issues, please use the issue tracker on the GitHub repository:
  https://github.com/wmayner/pyphi

For general discussion, you are welcome to join the pyphi-users group:
  https://groups.google.com/forum/#!forum/pyphi-users

To suppress this message, either:
  - Set `WELCOME_OFF: true` in your `pyphi_config.yml` file, or
  - Set the environment variable PYPHI_WELCOME_OFF to any value in your shell:
        export PYPHI_WELCOME_OFF='yes'



/Users/albantakis/miniconda3/envs/QIIT/lib/python3.10/site-packages/qutip/__init__.py:128: UserWarning: matplotlib not found: Graphics will not work.
  warnings.warn("matplotlib not found: Graphics will not work.")


In [2]:
%load_ext autoreload
%autoreload 2

# Define states and matrices

In [3]:
# Basis states (for both classical and quantum)
zero = basis(2,0)
one = basis(2,1)

# Normalized fully general quantum state
plus = (zero + one).unit()
minus = (zero - one).unit()
superposition = (1/3*zero + 2/3*one).unit()

# Density matrix representations of the states
rho_zero = zero * zero.dag() # could also be fock_dm(2,0)
rho_one = one * one.dag()
rho_plus = plus * plus.dag()
rho_minus = minus * minus.dag()
rho_sp = superposition * superposition.dag()
rho_mm = 0.5 * rho_zero + 0.5 * rho_one

rho_mix = 0.5*(tensor(rho_zero, rho_zero) + tensor(rho_one, rho_one))
rho_bell = ket2dm(bell_state(state = '00'))
rho_GHZ = ket2dm(ghz_state(N=3))
rho_W = ket2dm(w_state(N=3))
rho_bellmix = 1/3*(tensor(rho_bell, rho_zero) + tensor(rho_zero, rho_bell) + tensor(rho_zero, rho_bell).permute([1,0,2]))

rho_bennet = 1/(3-4)*(1-(tensor(rho_zero, rho_one, rho_plus) + tensor(rho_one, rho_plus, rho_zero) 
                        + tensor(rho_plus, rho_zero, rho_one) + tensor(rho_minus, rho_minus, rho_minus)))


In [4]:
pyphi.config.MEASURE = 'ID'
pyphi.config.PARTITION_TYPE = 'TRI'
pyphi.config.PICK_SMALLEST_PURVIEW = True

In [5]:
print('ID |0> : ', intrinsic_difference(rho_zero, rho_mm))
print('ID |+> : ', intrinsic_difference(rho_plus, rho_mm))

ID |0> :  (1.0, [array([1.+0.j, 0.+0.j])])
ID |+> :  (1.0, [array([0.70710678+0.j, 0.70710678+0.j])])


# Classical results

In [6]:
print(pyphi.config)

{ 'ACTUAL_CAUSATION_MEASURE': 'PMI',
  'ASSUME_CUTS_CANNOT_CREATE_NEW_CONCEPTS': False,
  'CACHE_POTENTIAL_PURVIEWS': True,
  'CACHE_REPERTOIRES': True,
  'CACHE_SIAS': False,
  'CACHING_BACKEND': 'fs',
  'CLEAR_SUBSYSTEM_CACHES_AFTER_COMPUTING_SIA': False,
  'CUT_ONE_APPROXIMATION': False,
  'FS_CACHE_DIRECTORY': '__pyphi_cache__',
  'FS_CACHE_VERBOSITY': 0,
  'LOG_FILE': 'pyphi.log',
  'LOG_FILE_LEVEL': 'INFO',
  'LOG_STDOUT_LEVEL': 'WARNING',
  'MAXIMUM_CACHE_MEMORY_PERCENTAGE': 50,
  'MEASURE': 'ID',
  'MONGODB_CONFIG': { 'collection_name': 'cache',
                      'database_name': 'pyphi',
                      'host': 'localhost',
                      'port': 27017},
  'NUMBER_OF_CORES': -1,
  'PARALLEL_COMPLEX_EVALUATION': False,
  'PARALLEL_CONCEPT_EVALUATION': False,
  'PARALLEL_CUT_EVALUATION': True,
  'PARTITION_TYPE': 'TRI',
  'PICK_SMALLEST_PURVIEW': True,
  'PRECISION': 6,
  'PRINT_FRACTIONS': True,
  'PROGRESS_BARS': True,
  'REDIS_CACHE': False,
  'REDIS_CONFIG':

In [7]:
qgate = cnot()
state = (1,1)
Qtpm = qgate.data.toarray()

In [8]:
Qtpm.real

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

In [9]:
# pyphi convention for state ordering is little endian
Qtpm = pyphi.convert.be2le_state_by_state(Qtpm.real)

In [10]:
network = pyphi.Network(Qtpm)
subsystem = pyphi.Subsystem(network, state)

In [11]:
ces = pyphi.compute.ces(subsystem)

In [12]:
ces

════════════════════════════════════════════════════════════
            Cause-effect structure (2 concepts)             
════════════════════════════════════════════════════════════
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  
              Concept: Mechanism = [n0], φ = 1              
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  
              MIC                         MIE               
  ┌──────────────────────────┐┌──────────────────────────┐  
  │  φ = 1                   ││  φ = 1                   │  
  │  Purview = [n0]          ││  Purview = [n0]          │  
  │  MIP:                    ││  MIP:                    │  
  │     ∅     ∅    n0        ││     ∅     ∅    n0        │  
  │    ─── ✕ ─── ✕ ───       ││    ─── ✕ ─── ✕ ───       │  
  │     ∅    n0     ∅        ││     ∅    n0     ∅        │  
  │  Repertoire:             ││  Repertoire:             │  
  │    ┌─────────────┐       ││    ┌─────────────┐       │  
  │    │ S    Pr(S)  │  

In [13]:
subsystem.find_mice(pyphi.direction.Direction(0), (1,))

Maximally-irreducible cause
  φ = 1/2
  Mechanism: [n1]
  Purview = [n0, n1]
  Direction: CAUSE
  MIP:
     ∅      ∅     n1 
    ─── ✕ ───── ✕ ───
     ∅    n0,n1    ∅ 
  Repertoire:
    ┌──────────────┐
    │ S     Pr(S)  │
    │ ╴╴╴╴╴╴╴╴╴╴╴╴ │
    │ 00    0      │
    │ 10    1/2    │
    │ 01    1/2    │
    │ 11    0      │
    └──────────────┘
  Partitioned repertoire:
    ┌──────────────┐
    │ S     Pr(S)  │
    │ ╴╴╴╴╴╴╴╴╴╴╴╴ │
    │ 00    1/4    │
    │ 10    1/4    │
    │ 01    1/4    │
    │ 11    1/4    │
    └──────────────┘

# Quantum results

## 2 Qubit

In [14]:
qgate = cnot()
#qgate = tensor(qeye(2),qeye(2),qeye(2)) 

In [15]:
rho = tensor(rho_minus, rho_plus)
#rho = tensor(rho_mm, rho_one)
#rho = rho_mix

In [16]:
rho

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[ 0.25  0.25 -0.25 -0.25]
 [ 0.25  0.25 -0.25 -0.25]
 [-0.25 -0.25  0.25  0.25]
 [-0.25 -0.25  0.25  0.25]]

In [17]:
rho_e = evolve_unitary(rho, qgate, 'effect')
rho_e

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 0.25  0.25 -0.25 -0.25]
 [ 0.25  0.25 -0.25 -0.25]
 [-0.25 -0.25  0.25  0.25]
 [-0.25 -0.25  0.25  0.25]]

In [18]:
rho_e.ptrace([1,])

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.5 0.5]
 [0.5 0.5]]

In [19]:
result = compute_ces(rho, qgate, direction = 'effect')

m:  (0,)
p:  (0,)  phi:  0.276692
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0.138346
p:  (0, 2)  phi:  0.138346
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0.069173
m:  (1,)
p:  (0,)  phi:  0
p:  (1,)  phi:  1.0
p:  (0, 1)  phi:  0.5
m:  (0, 1)
p:  (0,)  phi:  1.0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  1.0


In [21]:
result

[{'mech': (1,),
  'purview': (1,),
  'phi': 1.0,
  'mip':  ∅     ∅     1 
  ─── ✕ ─── ✕ ───
   ∅     1     ∅ ,
  'state': [array([0.70710678+0.j, 0.70710678+0.j])]},
 {'mech': (0, 1),
  'purview': (0,),
  'phi': 1.0,
  'mip':  ∅     ∅    0,1
  ─── ✕ ─── ✕ ───
   ∅     0     ∅ ,
  'state': [array([ 0.70710678+0.j, -0.70710678+0.j])]}]

In [22]:
result_cause = compute_ces(rho_e, qgate, direction = 'cause')

m:  (0,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  0.5
m:  (1,)
p:  (0,)  phi:  0
p:  (1,)  phi:  1.0
p:  (0, 1)  phi:  0.5
m:  (0, 1)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (0, 1)  phi:  1.0


In [23]:
result_cause

[{'mech': (0,),
  'purview': (0, 1),
  'phi': 0.5,
  'mip':  ∅     ∅     0 
  ─── ✕ ─── ✕ ───
   ∅    0,1    ∅ ,
  'state': [array([-0.70710678-0.j, -0.        -0.j, -0.        -0.j,  0.70710678+0.j]),
   array([ 0.        +0.j,  0.70710678+0.j, -0.70710678+0.j, -0.        +0.j])]},
 {'mech': (1,),
  'purview': (1,),
  'phi': 1.0,
  'mip':  ∅     ∅     1 
  ─── ✕ ─── ✕ ───
   ∅     1     ∅ ,
  'state': [array([0.70710678-0.j, 0.70710678+0.j])]},
 {'mech': (0, 1),
  'purview': (0, 1),
  'phi': 1.0,
  'mip':  ∅     0     1 
  ─── ✕ ─── ✕ ───
   ∅    0,1    ∅ ,
  'state': [array([ 0.5-0.j,  0.5+0.j, -0.5-0.j, -0.5-0.j])]}]

In [31]:
(ket2dm(Qobj(result_cause[0]['state'][0])) + ket2dm(Qobj(result_cause[0]['state'][1]))).unit()

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.25  0.    0.   -0.25]
 [ 0.    0.25 -0.25  0.  ]
 [ 0.   -0.25  0.25  0.  ]
 [-0.25  0.    0.    0.25]]

In [33]:
(tensor(rho_minus, rho_plus) + tensor(rho_plus, rho_minus)).unit()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.25  0.    0.   -0.25]
 [ 0.    0.25 -0.25  0.  ]
 [ 0.   -0.25  0.25  0.  ]
 [-0.25  0.    0.    0.25]]

In [35]:
(tensor(rho_minus, rho_minus) - tensor(rho_plus, rho_plus)).unit()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.   -0.25 -0.25  0.  ]
 [-0.25  0.    0.   -0.25]
 [-0.25  0.    0.   -0.25]
 [ 0.   -0.25 -0.25  0.  ]]

In [36]:
operator_to_vector(tensor(rho_minus, rho_plus))

Quantum object: dims = [[[2, 2], [2, 2]], [1]], shape = (16, 1), type = operator-ket
Qobj data =
[[ 0.25]
 [ 0.25]
 [-0.25]
 [-0.25]
 [ 0.25]
 [ 0.25]
 [-0.25]
 [-0.25]
 [-0.25]
 [-0.25]
 [ 0.25]
 [ 0.25]
 [-0.25]
 [-0.25]
 [ 0.25]
 [ 0.25]]

In [161]:
bell_state(state='01')

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[ 0.70710678]
 [ 0.        ]
 [ 0.        ]
 [-0.70710678]]

## 3 Qubits

In [80]:
qgate = tensor(qeye(2), cnot())
#qgate = tensor(qeye(2),qeye(2),qeye(2))

m:  (0,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (1,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (2,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (0, 1)
p:  (0,)  phi:  0
p:  (1,)  phi:  0.438722
p:  (2,)  phi:  0.438722
p:  (0, 1)  phi:  0.219361
p:  (0, 2)  phi:  0.219361
p:  (1, 2)  phi:  0.658083
p:  (0, 1, 2)  phi:  0.329041
m:  (0, 2)
p:  (0,)  phi:  0
p:  (1,)  phi:  0.438722
p:  (2,)  phi:  0.438722
p:  (0, 1)  phi:  0.219361
p:  (0, 2)  phi:  0.219361
p:  (1, 2)  phi:  0.658083
p:  (0, 1, 2)  phi:  0.329041
m:  (1, 2)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (0, 1, 2)
p:  (0,)  phi:  1.0


In [81]:
#rho = tensor(rho_bell, rho_zero)
rho = rho_GHZ
#rho = ket2dm(GHZ2)
rho

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.5 0.  0.  0.  0.  0.  0.  0.5]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.  0.  0.  0.  0.5]]

In [82]:
rho_e = evolve_unitary(rho, qgate, 'effect')
rho_e

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0.5 0.  0.  0.  0.  0.  0.5 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.  0.  0.  0.5 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]]

In [83]:
rho_e.ptrace([0,1])

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.5 0.  0.  0.5]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.5 0.  0.  0.5]]

In [68]:
entanglement_partition(rho_e)

[(0, 1, 2)]

In [69]:
result = compute_ces(rho, qgate, direction = 'effect')

m:  (0,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (1,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (2,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (0, 1)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (0, 2)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (1, 2)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (0, 1, 2)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0.5
p:  (0, 2)  phi:  0.5
p:  (1

In [70]:
result

[{'mech': (0, 1, 2),
  'purview': (0, 1, 2),
  'phi': 3.0,
  'mip':  ∅      ∅     0,1,2
  ─── ✕ ───── ✕ ─────
   ∅    0,1,2     ∅  ,
  'state': [array([0.70710678+0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
          0.        +0.j, 0.        +0.j, 0.        +0.j, 0.70710678+0.j])]}]

In [43]:
result_cause = compute_ces(rho_e, qgate, direction = 'cause')

m:  (0,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (1,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (2,)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0.5
p:  (0, 1, 2)  phi:  0.25
m:  (0, 1)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0.5
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  1.0
m:  (0, 2)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (1, 2)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p:  (1, 2)  phi:  0
p:  (0, 1, 2)  phi:  0
m:  (0, 1, 2)
p:  (0,)  phi:  0
p:  (1,)  phi:  0
p:  (2,)  phi:  0
p:  (0, 1)  phi:  0
p:  (0, 2)  phi:  0
p

In [44]:
result_cause

[{'mech': (2,),
  'purview': (1, 2),
  'phi': 0.5,
  'mip':  ∅     ∅     2 
  ─── ✕ ─── ✕ ───
   ∅    1,2    ∅ ,
  'state': [array([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]),
   array([0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j])]},
 {'mech': (0, 1),
  'purview': (0, 1, 2),
  'phi': 1.0,
  'mip':  ∅      ∅     0,1
  ─── ✕ ───── ✕ ───
   ∅    0,1,2    ∅ ,
  'state': [array([0.70710678+0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
          0.        +0.j, 0.        +0.j, 0.        +0.j, 0.70710678+0.j]),
   array([0.        +0.j, 0.70710678+0.j, 0.        +0.j, 0.        +0.j,
          0.        +0.j, 0.        +0.j, 0.70710678+0.j, 0.        +0.j])]},
 {'mech': (0, 1, 2),
  'purview': (0, 1, 2),
  'phi': 1.0,
  'mip':  ∅     0,1     2 
  ─── ✕ ───── ✕ ───
   ∅    0,1,2    ∅ ,
  'state': [array([0.70710678+0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
          0.        +0.j, 0.        +0.j, 0.        +0.j, 0.70710678+0.j])]}]

In [112]:
rho_W.ptrace([0,])

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.66666667 0.        ]
 [0.         0.33333333]]

In [88]:
result_cause[1]['state'][1]

array([0.        +0.j, 0.70710678+0.j, 0.        +0.j, 0.        +0.j,
       0.        +0.j, 0.        +0.j, 0.70710678+0.j, 0.        +0.j])

In [85]:
state

Quantum object: dims = [[2, 2, 2, 2, 2, 2, 2, 2], [1, 1, 1, 1, 1, 1, 1, 1]], shape = (256, 1), type = ket
Qobj data =
[[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]


In [92]:
(tensor(rho_bell, rho_plus) + tensor(ket2dm((ket('00') - ket('11')).unit()), rho_minus)).unit()

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0.25 0.   0.   0.   0.   0.   0.   0.25]
 [0.   0.25 0.   0.   0.   0.   0.25 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.25 0.   0.   0.   0.   0.25 0.  ]
 [0.25 0.   0.   0.   0.   0.   0.   0.25]]

In [90]:
ket2dm((ket('00') - ket('11')).unit())

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.5  0.   0.  -0.5]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [-0.5  0.   0.   0.5]]

In [103]:
intrinsic_difference(rho_GHZ.ptrace([0,1]), rho_test) 

(0.5, [array([0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j])])

In [102]:
rho_test = 1/2 * ket2dm(ket('00')) + 1/4* ket2dm(ket('01')) + 1/4* ket2dm(ket('11'))